# README

This document can be run directly to solve the problem. It contains the key mathematical theory and corresponding code snippets. 

Also consider running `main.py` interactively to be able to check the solution, i.e. `python -i main.py`.

#### Load CP-SAT and itertools

In [1]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

from itertools import combinations

#### Problem setup
**Setup problem indexing**
* Cells are indexed 1 to 28.
* Each almost magic square consists of 9 indices.
* slices to check the sums are pre-generated for convenience.

In [2]:
# indexes
#      1,  2,  3
#      4,  5,  6,  7,  8
#  9, 10, 11, 12, 13, 14
# 15, 16, 17, 18, 19, 20
# 21, 22, 23, 24, 25
#         26, 27, 28 
#
# Square 1
sq1 = [
     1, 2, 3,
     4, 5, 6,
    10,11,12]
sq2 = [
     6, 7, 8,
    12,13,14,
    18,19,20]
sq3 = [
     9,10,11,
    15,16,17,
    21,22,23]
sq4 = [
    17,18,19,
    23,24,25,
    26,27,28]
sqs = [sq1, sq2, sq3, sq4]

horzs = [slice(3*i, 3*i+3  ) for i in range(3)] # horizontal
verts = [slice(  i,     9,3) for i in range(3)] # vertical
diags = [slice(  0,     9,4), # main diag
         slice(  2,     7,2)] # off diag


#### Setup variables:
* $x_{n} = v$ $\Leftrightarrow$ cell $n$ contains value $v$.
* $b_{n_1, n_2} = 1$ $\Leftrightarrow$ $x_{n_1}$ > $x_{n_2}$; 0 otherwise.

In [3]:
## Define and save variables
infinity = 1828 #solver.infinity()
nrange = [i+1 for i in range(28)]
x = {}
for n in nrange:
    x[n] = solver.IntVar(1., infinity, f'x[{n}]')
b = {}
for n1, n2 in combinations(nrange, 2):
        b[n1,n2] = solver.BoolVar(f'b[{n1},{n2}]')

#### Uniqueness constraint
No two $x_n$ can have the same value. We can do this using $b_{n_1, n_2}$ in conjunction with a big-$M$ penalty.
* $x_{n1} \geq x_{n2} + b_{n_1, n_2} - M(1-b_{n_1, n_2})$
* $x_{n2} \geq x_{n1} + (1-b_{n_1, n_2}) - Mb_{n_1, n_2}$

Note that if $x_{n1} > x_{n2}$, the constraints evaluate to:
* $x_{n1} \geq x_{n2} + 1 $
* $x_{n2} \geq x_{n1} - M$

It follows that $M$ should be much bigger than the largest possible value of $x$.

In [4]:
## Uniqueness constraints
M = 1828 # max M
for n1, n2 in combinations(nrange, 2):
        solver.Add(x[n1] >= x[n2] +    b[n1,n2]  - M*(1-b[n1,n2]))
        solver.Add(x[n2] >= x[n1] + (1-b[n1,n2]) - M*   b[n1,n2] )


### Sum constraints (for each almost magic square)
* For each square, we first create expressions for all horizontal, diagonal sums. 
* Given any two sum, the absolute value of their difference must be no more than 1. While the absolute value function is nonlinear, we can write the constraint piece-wise, i.e. in two parts:
    * $sum_1 - sum_2 \leq 1$
    * $sum_1 - sum_2 \geq -1$
* We should also not bother looking for worse solutions than the example one
    * $\sum_n x_n \leq 1828$

In [5]:
## Sums constraints
for sq in sqs:
    sums = [sum(x[n] for n in sq[sum_slice]) for sum_slice in horzs + verts + diags]
    for sum1, sum2 in combinations(sums, 2):
        solver.Add(sum1 - sum2 <=  1)
        solver.Add(sum1 - sum2 >= -1)

solver.Add(sum(x[n] for n in nrange) <= 1828)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x110425150> >

#### Objective function
Minimize $\sum_n x_n$

In [6]:
solver.Minimize(sum(x[n] for n in nrange))
status = solver.Solve()

In [7]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    X = []
    for n in nrange:
        X.append(x[n].solution_value())
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 470.0000000000071


In [8]:
X

[26.0,
 2.0,
 34.0,
 29.0,
 21.0,
 13.0,
 4.0,
 18.0,
 24.0,
 7.0,
 39.0,
 16.0,
 12.0,
 6.0,
 37.0,
 23.0,
 9.0,
 5.0,
 19.0,
 10.0,
 8.0,
 40.0,
 22.0,
 11.0,
 1.0,
 3.0,
 17.0,
 14.0]